# Задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd
import os
import time
import re

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [4]:
%cd gdrive/My\ Drive/work/GB_NLP/9_2

[Errno 2] No such file or directory: 'gdrive/My Drive/work/GB_NLP/9_2'
/content


In [5]:
!pwd

/content


Взял для обучения четверостишья Губермана

In [6]:
!wget https://raw.githubusercontent.com/drafterleo/pie-poem/master/data/gariki.txt

--2022-06-28 14:16:39--  https://raw.githubusercontent.com/drafterleo/pie-poem/master/data/gariki.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 688530 (672K) [text/plain]
Saving to: ‘gariki.txt’

gariki.txt          100%[===================>] 672.39K  --.-KB/s    in 0.03s   

2022-06-28 14:16:40 (24.9 MB/s) - ‘gariki.txt’ saved [688530/688530]



In [7]:
!ls *.txt

gariki.txt


In [8]:
path_to_file = 'gariki.txt'

In [9]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 386661 characters


In [10]:
print(text[:500])

Утучняется плоть,
Испаряется пыл.
Годы вышли на медленный ужин. 
И приятно подумать, 
Что все-таки был 
И кому-то бывал я нужен.

Мне Маркса жаль: его наследство
свалилось в русскую купель:
здесь цель оправдывала средства, и 
средства обосрали цель.

Во благо классу-гегемону, 
чтоб неослабно правил он,
во всякий миг доступен шмону 
отдельно взятый гегемон.

Слой человека в нас чуть-чуть 
наслоен зыбко и тревожно,
легко в скотину нас вернуть, 
поднять обратно очень сложно.

Я молодых, в остатках 


In [11]:
df = pd.DataFrame(text.split(sep='\n\n'))
df.head()

,0
0,"Утучняется плоть,\nИспаряется пыл.\nГоды вышли..."
1,Мне Маркса жаль: его наследство\nсвалилось в р...
2,"Во благо классу-гегемону, \nчтоб неослабно пра..."
3,Слой человека в нас чуть-чуть \nнаслоен зыбко ...
4,"Я молодых, в остатках сопель, \nбоюсь, трясущи..."


In [12]:
df = df.rename(columns={0: 'gariki'})
df.head()

,gariki
0,"Утучняется плоть,\nИспаряется пыл.\nГоды вышли..."
1,Мне Маркса жаль: его наследство\nсвалилось в р...
2,"Во благо классу-гегемону, \nчтоб неослабно пра..."
3,Слой человека в нас чуть-чуть \nнаслоен зыбко ...
4,"Я молодых, в остатках сопель, \nбоюсь, трясущи..."


In [13]:
def replacement_punkt(text, regex = re.compile(r'[^\w\s]')):
    try:
        return re.sub(regex, " ", text)
    except:
        return " "

In [14]:

def replacement(text, regex=re.compile('\\n')):
    try:
        return re.sub(regex, " \n ", text)
    except:
        return " "

Попробовал обозначать начало и конец четверостишья

In [15]:
def preprocess_text(text):
    text = str(text)
    text = text.lower()
    text = '<start> ' + text + ' <end>'

    return text

In [16]:
df['gariki_next'] = df['gariki'].apply(replacement)

In [17]:
df['gariki_next'] = df['gariki_next'].apply(replacement_punkt)

In [18]:
df['gariki_next'] = df['gariki_next'].apply(preprocess_text)

In [19]:
df['gariki_next'][0]

'<start> утучняется плоть  \n испаряется пыл  \n годы вышли на медленный ужин   \n и приятно подумать   \n что все таки был  \n и кому то бывал я нужен  <end>'

In [20]:
df.head()

,gariki,gariki_next
0,"Утучняется плоть,\nИспаряется пыл.\nГоды вышли...",<start> утучняется плоть \n испаряется пыл \...
1,Мне Маркса жаль: его наследство\nсвалилось в р...,<start> мне маркса жаль его наследство \n сва...
2,"Во благо классу-гегемону, \nчтоб неослабно пра...",<start> во благо классу гегемону \n чтоб нео...
3,Слой человека в нас чуть-чуть \nнаслоен зыбко ...,<start> слой человека в нас чуть чуть \n насл...
4,"Я молодых, в остатках сопель, \nбоюсь, трясущи...",<start> я молодых в остатках сопель \n боюс...


In [21]:
len(df['gariki_next'])

3322

In [22]:
df['gariki_next'][0].split()

['<start>',
 'утучняется',
 'плоть',
 'испаряется',
 'пыл',
 'годы',
 'вышли',
 'на',
 'медленный',
 'ужин',
 'и',
 'приятно',
 'подумать',
 'что',
 'все',
 'таки',
 'был',
 'и',
 'кому',
 'то',
 'бывал',
 'я',
 'нужен',
 '<end>']

In [23]:
list_for_vocab = []
for i in range(len(df['gariki_next'])):
    for token in df['gariki_next'][i].split():
        list_for_vocab.append(token)

In [24]:
vocab = sorted(set(list_for_vocab))

In [25]:
len(vocab)

19137

In [26]:
token2idx = {u:i for i, u in enumerate(vocab)}
idx2token = np.array(vocab)

In [27]:
text_as_int = np.array([token2idx[c] for c in list_for_vocab])

In [28]:
text_as_int[:50], list_for_vocab[:50], len(text_as_int), len(list_for_vocab)

(array([    3, 17811, 10953,  5528, 13065,  2773,  2481,  7976,  7275,
        17349,  5181, 12541, 11296, 18644,  2131, 16345,  1060,  5181,
         6121, 16674,  1050, 19045,  9215,     2,     3,  7550,  7200,
         4128,  4016,  8314, 14140,  1098, 13986,  6552,  4964, 18387,
         9857, 15649,  5181, 15649,  9421, 18387,     2,     3,  1720,
          590,  5924,  2567, 18645,  8791]),
 ['<start>',
  'утучняется',
  'плоть',
  'испаряется',
  'пыл',
  'годы',
  'вышли',
  'на',
  'медленный',
  'ужин',
  'и',
  'приятно',
  'подумать',
  'что',
  'все',
  'таки',
  'был',
  'и',
  'кому',
  'то',
  'бывал',
  'я',
  'нужен',
  '<end>',
  '<start>',
  'мне',
  'маркса',
  'жаль',
  'его',
  'наследство',
  'свалилось',
  'в',
  'русскую',
  'купель',
  'здесь',
  'цель',
  'оправдывала',
  'средства',
  'и',
  'средства',
  'обосрали',
  'цель',
  '<end>',
  '<start>',
  'во',
  'благо',
  'классу',
  'гегемону',
  'чтоб',
  'неослабно'],
 68634,
 68634)

### train and target

In [29]:
seq_list = []
for i in range(len(df['gariki_next'])):
    seq_list.append(len(df['gariki_next'][i].split()))

In [30]:
max(seq_list)

59

In [31]:
min(seq_list)

9

In [32]:
sum(seq_list)/len(seq_list)

20.66044551475015

In [33]:
# The maximum length sentence you want for a single input in characters
seq_length = 40
examples_per_epoch = len(text_as_int)//(seq_length+1)

# Create training examples / targets
token_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in token_dataset.take(5):
    print(idx2token[i.numpy()])

<start>
утучняется
плоть
испаряется
пыл


In [34]:
sequences = token_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join(idx2token[item.numpy()])))

'<start> утучняется плоть испаряется пыл годы вышли на медленный ужин и приятно подумать что все таки был и кому то бывал я нужен <end> <start> мне маркса жаль его наследство свалилось в русскую купель здесь цель оправдывала средства и средства обосрали'
'цель <end> <start> во благо классу гегемону чтоб неослабно правил он во всякий миг доступен шмону отдельно взятый гегемон <end> <start> слой человека в нас чуть чуть наслоен зыбко и тревожно легко в скотину нас вернуть поднять обратно очень сложно <end>'
'<start> я молодых в остатках сопель боюсь трясущих жизнь как грушу в душе темно у них как в жопе а в жопе зуд потешить душу <end> <start> когда истории сквозняк свистит по душам и державам один ползет в нору слизняк другой'
'вздувается удавом <end> <start> добро не отвергая средства зла по ним и пожинает результаты в раю где применяется смола архангелы копытны и рогаты <end> <start> по крови проникая до корней пронизывая воздух небосвода неволя растлевает нас сильней чем самая беспут

In [35]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [36]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2token[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2token[target_example.numpy()])))

Input data:  '<start> утучняется плоть испаряется пыл годы вышли на медленный ужин и приятно подумать что все таки был и кому то бывал я нужен <end> <start> мне маркса жаль его наследство свалилось в русскую купель здесь цель оправдывала средства и средства'
Target data: 'утучняется плоть испаряется пыл годы вышли на медленный ужин и приятно подумать что все таки был и кому то бывал я нужен <end> <start> мне маркса жаль его наследство свалилось в русскую купель здесь цель оправдывала средства и средства обосрали'


In [37]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 3000

dataset = dataset.shuffle(BUFFER_SIZE, seed=21).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 40), dtype=tf.int64, name=None), TensorSpec(shape=(64, 40), dtype=tf.int64, name=None))>

In [38]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 2048

# Number of RNN units
rnn_units = 4096

In [39]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [40]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [41]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [42]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [43]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [44]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 40, 19137) # (batch_size, sequence_length, vocab_size)


In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 2048)        39192576  
                                                                 
 gru_2 (GRU)                 (None, None, 4096)        75522048  
                                                                 
 gru_3 (GRU)                 (None, None, 4096)        100687872 
                                                                 
 gru_4 (GRU)                 (None, None, 4096)        100687872 
                                                                 
 dense_2 (Dense)             (None, None, 19137)       78404289  
                                                                 
Total params: 394,494,657
Trainable params: 394,494,657
Non-trainable params: 0
_________________________________________________________________


In [46]:
example_batch_predictions[0]

<tf.Tensor: shape=(40, 19137), dtype=float32, numpy=
array([[-2.8247799e-04,  5.8257574e-04, -4.1446462e-04, ...,
        -8.4931822e-04, -1.9034298e-04,  3.7484785e-04],
       [-5.8326131e-04,  1.4189158e-03, -2.2854861e-04, ...,
        -1.8075497e-03,  5.0783140e-04,  6.8543729e-04],
       [-7.0876552e-04,  1.0629648e-03,  3.0784251e-04, ...,
        -1.5513515e-03,  1.1262735e-03,  9.6897804e-04],
       ...,
       [ 4.9388484e-04, -4.3567090e-04, -2.1847798e-03, ...,
        -1.4940186e-03, -4.0068091e-03, -1.6869081e-03],
       [ 3.4510085e-05, -8.6202373e-04, -2.7521439e-03, ...,
        -9.7661139e-04, -2.3199916e-03, -1.7621771e-03],
       [-1.2133375e-03, -2.0367578e-03, -1.8232554e-03, ...,
        -7.5757684e-04, -1.0999202e-03, -2.6829913e-04]], dtype=float32)>

In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [48]:
print("Input: \n", repr(" ".join(idx2token[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2token[sampled_indices ])))

Input: 
 'лаком я в мысли простой утвердился с годами семья это тайна покрытая браком <end> <start> из некоего жизненного круга нам выйти с неких пор не удаётся поэтому случайная подруга нечаянная влага из колодца <end> <start> ведём ли мы беседы грустные'

Next Char Predictions: 
 'кормил гусь национальный прибавили каждую утоленья свежих знавал девка антисемит цинизм тупик граница нести рожи дремлю вытворит суйся работящая поединке переврет стремились местах цветут копать жду глух свистнул глажу норы хохочет небосвода фактур веков кутерьма кошмары вляпывался кишении стайку сядем'


## Train the model

In [49]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 40, 19137)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.859294


In [50]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

### Configure checkpoints

In [51]:
!rm -rf ./training_checkpoints

In [52]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [53]:
!ls

gariki.txt  gdrive  sample_data


In [54]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    period=5,
    #save_freq=88*5,
    save_weights_only=True)

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stop = EarlyStopping(monitor='loss',
                                     min_delta=0.003,
                                     patience=1,
                                     verbose=1,
                                     mode='auto',
                                     baseline=None,
                                     restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.1,
                                        patience=0,
                                        verbose=1,
                                        mode='auto',
                                        min_delta=0.001,
                                        cooldown=0,
                                        min_lr=1e-10)

### Execute the training

In [55]:
EPOCHS = 120

Остановил обучение когда исчезла надежда на хэппи энд.

In [56]:
for i in range(EPOCHS):
    model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])
    model.reset_states()

26/26 [==============================] - ETA: 0s - loss: 4.9697 - accuracy: 0.1897
Epoch 1: loss improved from inf to 4.96968, saving model to ./training_checkpoints/ckpt_1
26/26 [==============================] - ETA: 0s - loss: 0.5980 - accuracy: 0.8818
Epoch 1: loss improved from 4.96968 to 0.59799, saving model to ./training_checkpoints/ckpt_1
26/26 [==============================] - ETA: 0s - loss: 0.0996 - accuracy: 0.9828
Epoch 1: loss improved from 0.59799 to 0.09964, saving model to ./training_checkpoints/ckpt_1
26/26 [==============================] - ETA: 0s - loss: 0.0642 - accuracy: 0.9881
Epoch 1: loss improved from 0.09964 to 0.06424, saving model to ./training_checkpoints/ckpt_1
26/26 [==============================] - ETA: 0s - loss: 0.0591 - accuracy: 0.9877
Epoch 1: loss improved from 0.06424 to 0.05911, saving model to ./training_checkpoints/ckpt_1
26/26 [==============================] - ETA: 0s - loss: 0.0645 - accuracy: 0.9870
Epoch 1: loss did not improve from 0

KeyboardInterrupt: ignored

In [ ]:
#history = model.fit(dataset, 
#                    epochs=EPOCHS,  
#                    callbacks=[checkpoint_callback, reduce_lr])

## Generate text

In [84]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_1'

In [85]:
checkpoint_dir

'./training_checkpoints'

In [86]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [87]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 2048)        39192576  
                                                                 
 gru_5 (GRU)                 (None, None, 4096)        75522048  
                                                                 
 gru_6 (GRU)                 (None, None, 4096)        100687872 
                                                                 
 gru_7 (GRU)                 (None, None, 4096)        100687872 
                                                                 
 dense_3 (Dense)             (None, None, 19137)       78404289  
                                                                 
Total params: 394,494,657
Trainable params: 394,494,657
Non-trainable params: 0
_________________________________________________________________


In [88]:
def generate_text(model, temperature, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 70

    # Converting our start string to numbers (vectorizing)
    input_eval = [token2idx[s] for s in start_string.lower().split()]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature # для понятности

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2token[predicted_id])

    return (start_string + ' '.join(text_generated))

In [94]:
text_ = generate_text(model, 0.7, start_string=u"<start> Жизнь ")
print(text_)

<start> Жизнь не внемлю лёгкого поведения есть закон рива ударит бы варяг питает игре в этой купле и я вперед талии чем еврей кто знает каждый забытое в жизни театральность <start> всё что прозрачный наживлённый этим любой способа жизни театральность <start> незря <start> поскольку денежный расти по счёту итога на белом а если взглянуть общения <end> <start> доволен я знаю что к вам делясь где то словно беда во рту рассветом нельзя


In [90]:
len(text_)

399

In [96]:
text_ = generate_text(model, 0.5, start_string=u"<start> Жизнь ")
print(text_)

<start> Жизнь не влился я не внемлю талии чем еврей кто знает каждый разум <end> <start> не на что будущее монахов и на белом мир на белом меня простишь весь слышны обычно весь дрязг этом нельзя и говорю что хочется при этом нельзя и на белом меня как зонтом и назад и на мне подкатит и тем еврею меж жизни театральность омерзения еврейского искусства себя хотящие течет шеи всего моя стонет и


In [98]:
text_ = generate_text(model, 0.3, start_string=u"<start> Жизнь ")
print(text_)

<start> Жизнь не судьба моя стонет и на белом меня простишь весь слышны обычно весь слышны обычно весь слышны обычно весь слышны обычно весь дрязг этом нельзя и безмерно знает каждый забытое итога на хер чтоб в этой купле и что мне подкатит и дикое множество отличен лучшей сквозь стены <start> я не воспеты к вам делясь где то совал жизни театральность <start> я не влился я на белом мир на белом


In [104]:
text_ = generate_text(model, 0.2, start_string=u"<start> Жизнь ")
print(text_)

<start> Жизнь не судьба моя стонет и на белом а рот враг <end> <start> я не воспеты по разному мне подкатит и на белом а ебутся <end> <start> я не воспеты к ебене ясновидца я не находят верша так ли россию в себе по разному у каждого жалею во мне подкатит и на белом а я в этой купле и я в этой купле и суками вокруг пламень я не находят верша


Вывод: чем меньше температура тем больше матов :-)